In [1]:
import pandas as pd
import numpy as np
import duckdb
from fila_estacoes_utils import P_1 as P

ouro_conn = duckdb.connect('ouro_db')
modelling_conn = duckdb.connect('modelling_db')

# ABT Estações

In [3]:
abt_estacoes_df = ouro_conn.execute(
"""
SELECT DISTINCT
    precipitacao.id_estacao
    ,dim_estacoes.latitude
    ,dim_estacoes.longitude
    ,dim_estacoes.vl_declividade
    ,dim_estacoes.vl_altitude
    ,dim_estacoes.vl_distancia_oceano
    ,dim_estacoes.vl_aspecto_relevo
    ,precipitacao.dt_medicao
    ,precipitacao.vl_precipitacao
    ,temperatura_maxima.vl_temperatura_maxima
    ,temperatura_media.vl_temperatura_media
    ,temperatura_minima.vl_temperatura_minima
    ,umidade_maxima.vl_umidade_relativa_maxima
    ,umidade_media.vl_umidade_relativa_media
    ,umidade_minima.vl_umidade_relativa_minima
    ,vento_2m_max.vl_velocidade_vento_2m_maxima
    ,vento_2m_media.vl_velocidade_vento_2m_media
    ,vento_10m_media.vl_velocidade_vento_10m_media
FROM
    fato_estacoes_precipitacao AS precipitacao
LEFT JOIN
    fato_estacoes_temperatura_maxima AS temperatura_maxima
    ON temperatura_maxima.id_estacao = precipitacao.id_estacao
    AND temperatura_maxima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_temperatura_media AS temperatura_media
    ON temperatura_media.id_estacao = precipitacao.id_estacao
    AND temperatura_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_temperatura_minima AS temperatura_minima
    ON temperatura_minima.id_estacao = precipitacao.id_estacao
    AND temperatura_minima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_maxima AS umidade_maxima
    ON umidade_maxima.id_estacao = precipitacao.id_estacao
    AND umidade_maxima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_media AS umidade_media
    ON umidade_media.id_estacao = precipitacao.id_estacao
    AND umidade_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_minima AS umidade_minima
    ON umidade_minima.id_estacao = precipitacao.id_estacao
    AND umidade_minima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_2m_maxima AS vento_2m_max
    ON vento_2m_max.id_estacao = precipitacao.id_estacao
    AND vento_2m_max.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_2m_media AS vento_2m_media
    ON vento_2m_media.id_estacao = precipitacao.id_estacao
    AND vento_2m_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_10m_media AS vento_10m_media
    ON vento_10m_media.id_estacao = precipitacao.id_estacao
    AND vento_10m_media.dt_medicao = precipitacao.dt_medicao
JOIN dim_estacoes
    ON precipitacao.id_estacao = dim_estacoes.id_estacao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes AS
SELECT * FROM abt_estacoes_df
""")

ouro_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes AS
SELECT * FROM abt_estacoes_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# ABT Produtos

In [2]:
abt_produtos_df = ouro_conn.execute(
"""
SELECT DISTINCT
   abt_estacoes.*
    ,chirps.vl_precipitacao AS vl_precipitacao_chirps
    ,cpc.vl_precipitacao AS vl_precipitacao_cpc
    ,cpc.vl_temperatura_maxima AS vl_temperatura_maxima_cpc
    ,cpc.vl_temperatura_minima AS vl_temperatura_minima_cpc
    ,gpm_final_run.vl_precipitacao AS vl_precipitacao_gpm_final_run
    ,gpm_late_run.vl_precipitacao AS vl_precipitacao_gpm_late_run
    ,power.vl_precipitacao AS vl_precipitacao_power
    ,power.vl_temperatura_maxima_2m_K AS vl_temperatura_maxima_2m_K_power
    ,power.vl_temperatura_media_2m_K AS vl_temperatura_media_2m_K_power
    ,power.vl_temperatura_minima_2m_K AS vl_temperatura_minima_2m_K_power
    ,power.vl_umidade_relativa_2m AS vl_umidade_relativa_2m_power
    ,power.vl_pressao_nivel_superficie AS vl_pressao_nivel_superficie_power
    ,power.vl_irradiancia_allsky AS vl_irradiancia_allsky_power
    ,power.vl_direcao_vento_10m AS vl_direcao_vento_10m_power
    ,power.vl_direcao_vento_2m AS vl_direcao_vento_2m_power
    ,power.vl_temperatura_orvalho_2m_K AS vl_temperatura_orvalho_2m_K_power
    ,power.vl_vento_10m AS vl_vento_10m_power
    ,power.vl_vento_medio_2m AS vl_vento_medio_2m_power
    ,power.vl_vento_maximo_2m AS vl_vento_maximo_2m_power
    ,power.vl_vento_maximo_10m AS vl_vento_maximo_10m_power
FROM abt_estacoes
JOIN fato_estacoes_latlon_produtos_df AS latlon
    ON abt_estacoes.id_estacao = latlon.id_estacao
LEFT JOIN fato_produto_chirps AS chirps
    ON latlon.lat_chirps = chirps.lat
    AND latlon.lon_chirps = chirps.lon
    AND abt_estacoes.dt_medicao = chirps.dt_medicao
LEFT JOIN fato_produto_cpc AS cpc
    ON latlon.lat_cpc = cpc.lat
    AND latlon.lon_cpc = cpc.lon
    AND abt_estacoes.dt_medicao = cpc.dt_medicao
LEFT JOIN fato_produto_gpm_final_run AS gpm_final_run
    ON latlon.lat_gpm_final_run = gpm_final_run.lat
    AND latlon.lon_gpm_final_run = gpm_final_run.lon
    AND abt_estacoes.dt_medicao = gpm_final_run.dt_medicao
LEFT JOIN fato_produto_gpm_late_run AS gpm_late_run
    ON latlon.lat_gpm_late_run = gpm_late_run.lat
    AND latlon.lon_gpm_late_run = gpm_late_run.lon
    AND abt_estacoes.dt_medicao = gpm_late_run.dt_medicao
LEFT JOIN fato_produto_power AS power
    ON latlon.lat_power = power.lat
    AND latlon.lon_power = power.lon
    AND abt_estacoes.dt_medicao = power.dt_medicao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_produtos AS
SELECT * FROM abt_produtos_df
""")

ouro_conn.execute("""
CREATE OR REPLACE TABLE abt_produtos AS
SELECT * FROM abt_produtos_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))